### Imports

In [31]:
import requests
import pandas as pd
from datetime import datetime

### Authentication

In [13]:
url = "https://api.personio.de/v1/auth"
payload = {
    "client_id": "papi-9cecb510-8b07-44f6-b3ca-02dabe0615ae",
    "client_secret": "papi-NGYwMjc2MGMtODBjYi00N2EyLThmNzItN2VhNGZjNDkzOTRh"
}

response = requests.post(url, json=payload)
response_data = response.json()

if response.status_code == 200:
    auth_token = response_data['data']['token'] if 'data' in response_data and 'token' in response_data['data'] else None
    if auth_token:
        print("Auth token:", auth_token)
    else:
        print("Token not found in the response.")
else:
    print("Failed to authenticate:", response.text)


Auth token: papi-wMfNaPhJuhvyO1JAD6tsF4phF0osArdqN-C2IBiOp90SHP2BEb_k7jWIfQ7D8Lu3ufCbOPxOs7vziHgcNc6RGg-MTExMTEz


### Get List of All employees

In [19]:
url = "https://api.personio.de/v1/company/employees"
headers = {
    "Authorization": "Bearer " + auth_token  # Replace with your actual auth token
}

response = requests.get(url, headers=headers)
response_data = response.json()

if response.status_code == 200:
    print("Success")  # Prints the JSON response
else:
    print("Error:", response.status_code, response.text)
    
    
# For now were only interested in the full names and ids of active employees. Lets create two lists:

names = []
ids = []

if response_data.get('success') and 'data' in response_data:
    for employee in response_data['data']:
        # Check if the employee status is active
        if employee['attributes']['status']['value'] == 'active':
            # Concatenate first name and last name
            name = employee['attributes']['first_name']['value'] + " " + employee['attributes']['last_name']['value']
            names.append(name)

            # Get the ID
            id = employee['attributes']['id']['value']
            ids.append(id)

print("Names:", names)
print("IDs:", ids)

Success
Names: ['Joanna Williams', 'Mira Ebinger', 'Christopher Ball', 'Zoltan Szabo', 'Daniel Baur', 'Iris Gallmann', 'Markus Weber', 'Elias Styner', 'Bruno Studer', 'Damian Steiner', 'Beat Steiner', 'Steffen Schneider', 'Felix Schlegel', 'Niclas Naeff', 'Markus Limacher', 'Emanuel Kohler', 'Taryn Kiser', 'Johnny Luu', 'Diana Enggist', 'Matthias Engels', 'Martin Eisenhammer', 'Urs Aeschbacher', 'Erika Lang', 'Lukas Barwitzki']
IDs: [19340582, 18500033, 18499814, 18228492, 17706734, 17632134, 12487208, 12487207, 12487206, 12487205, 12487204, 12487202, 12487201, 12487200, 12487197, 12487196, 12487194, 12487193, 12487188, 12487187, 12487186, 12288221, 12288219, 11419365]


### Get Attendances

In [28]:
ids = [12487201]

In [29]:
url = "https://api.personio.de/v1/company/attendances"
headers = {
    "Authorization": "Bearer " + auth_token  # Replace with your actual auth token
}
params = {
    "start_date": "2023-11-01",
    "end_date": "2023-11-30",
    "employees": ids  # This is the list of active employee IDs you obtained earlier
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    attendances = response.json()
    print(attendances)
else:
    print("Error:", response.status_code, response.text)


{'success': True, 'metadata': {'total_elements': 35, 'current_page': 0, 'total_pages': 1}, 'data': [{'id': 271605189, 'type': 'AttendancePeriod', 'attributes': {'employee': 12487201, 'date': '2023-11-08', 'start_time': '09:30', 'end_time': '12:00', 'break': 0, 'comment': '', 'updated_at': '2023-11-30T15:46:48+00:00', 'status': 'confirmed', 'project': {'type': 'Project', 'id': 302257, 'attributes': {'name': 'BAKOM', 'active': True}}, 'is_holiday': False, 'is_on_time_off': False}}, {'id': 271605192, 'type': 'AttendancePeriod', 'attributes': {'employee': 12487201, 'date': '2023-11-08', 'start_time': '12:55', 'end_time': '16:00', 'break': 0, 'comment': '', 'updated_at': '2023-11-30T15:46:48+00:00', 'status': 'confirmed', 'project': {'type': 'Project', 'id': 302257, 'attributes': {'name': 'BAKOM', 'active': True}}, 'is_holiday': False, 'is_on_time_off': False}}, {'id': 272347789, 'type': 'AttendancePeriod', 'attributes': {'employee': 12487201, 'date': '2023-11-30', 'start_time': '13:00', 'e

In [34]:
attendances

{'success': True,
 'metadata': {'total_elements': 35, 'current_page': 0, 'total_pages': 1},
 'data': [{'id': 271605189,
   'type': 'AttendancePeriod',
   'attributes': {'employee': 12487201,
    'date': '2023-11-08',
    'start_time': '09:30',
    'end_time': '12:00',
    'break': 0,
    'comment': '',
    'updated_at': '2023-11-30T15:46:48+00:00',
    'status': 'confirmed',
    'project': {'type': 'Project',
     'id': 302257,
     'attributes': {'name': 'BAKOM', 'active': True}},
    'is_holiday': False,
    'is_on_time_off': False}},
  {'id': 271605192,
   'type': 'AttendancePeriod',
   'attributes': {'employee': 12487201,
    'date': '2023-11-08',
    'start_time': '12:55',
    'end_time': '16:00',
    'break': 0,
    'comment': '',
    'updated_at': '2023-11-30T15:46:48+00:00',
    'status': 'confirmed',
    'project': {'type': 'Project',
     'id': 302257,
     'attributes': {'name': 'BAKOM', 'active': True}},
    'is_holiday': False,
    'is_on_time_off': False}},
  {'id': 27234

In [37]:
# Create a Dataframe out of this

# Sample response data (replace with your actual data)
response_data = response.json()['data']

# Initialize an empty list for storing data
data = []

# Process each attendance record
for record in response_data:
    project_info = record['attributes']['project']
    project_name = 'Sonstiges' if project_info is None else project_info['attributes']['name']
    date = record['attributes']['date']
    start_time = datetime.strptime(record['attributes']['start_time'], '%H:%M')
    end_time = datetime.strptime(record['attributes']['end_time'], '%H:%M')
    attendance_time = (end_time - start_time).total_seconds() / 3600  # Convert to hours

    # Add to data list
    data.append({'Date': date, 'Project': project_name, 'AttendanceTime': attendance_time})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Group by Date and Project, and sum the Attendance Time
df_summary = df.groupby(['Date', 'Project']).sum().unstack(fill_value=0)

df_summary['Total'] = df_summary.sum(axis=1)

df_summary

AttendanceTime               Total
Project             BAKOM Sonstiges          
Date                                         
2023-11-01       5.583333  0.000000  5.583333
2023-11-02       5.583333  0.000000  5.583333
2023-11-03       5.583333  0.000000  5.583333
2023-11-06       5.583333  0.000000  5.583333
2023-11-07       5.583333  0.000000  5.583333
2023-11-08       5.583333  0.000000  5.583333
2023-11-09       5.583333  0.000000  5.583333
2023-11-10       5.583333  0.000000  5.583333
2023-11-13       5.583333  0.000000  5.583333
2023-11-14       5.583333  0.000000  5.583333
2023-11-15       5.583333  0.000000  5.583333
2023-11-16       5.583333  0.000000  5.583333
2023-11-17       2.566667  3.016667  5.583333
2023-11-27       1.000000  4.583333  5.583333
2023-11-28       1.000000  4.583333  5.583333
2023-11-29       1.000000  4.583333  5.583333
2023-11-30       2.583333  3.000000  5.583333